In [ ]:
!git clone https://github.com/google/model_search.git

Cloning into 'model_search'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 203 (delta 6), reused 22 (delta 4), pack-reused 170
Receiving objects: 100% (203/203), 252.61 KiB | 2.10 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [ ]:
%cd model_search
!pwd

In [ ]:
!ls

In [ ]:
!pip install -r requirements.txt

In [ ]:
!ls model_search

In [ ]:
!protoc --python_out=./ model_search/proto/phoenix_spec.proto
!protoc --python_out=./ model_search/proto/hparam.proto
!protoc --python_out=./ model_search/proto/distillation_spec.proto
!protoc --python_out=./ model_search/proto/ensembling_spec.proto
!protoc --python_out=./ model_search/proto/transfer_learning_spec.proto

In [ ]:
# Cargar autenticación de Google para poder cargar los archivos
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Importación de librerías y dependencias

from sklearn import preprocessing
import pandas as pd
import os
import numpy as np
from sklearn import metrics
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from keras.models import Sequential
#from keras.layers.core import Dense, Activation
#from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, zero_one_loss
from scipy.stats import zscore
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64

#-------------------------------------------------------------------
from matplotlib import pyplot as plt
from sklearn.svm import SVC

In [ ]:
# Carga de todos los csv que componen el CIC-IDS-2018 a un solo DataFrame de Pandas

ids = ['1qcnBAkJzZlLMHxeQztEJ8pu9h438OksU','1Xz0jLd8dkUiifOFEFvCxGsnsF1N07zOr','1Xz0jLd8dkUiifOFEFvCxGsnsF1N07zOr','1QotaesVIgNqisZrf2fFDDSSo0Bkw2jIC', '1c7eq8zKDN6v3y4yf9gbrOH0K-fOONlb4','1BQCLnXOtyaQueNGKYgeEHo5oXvo2iNso','1IaKsgZ3ccZXC4cTwe5oX5FvE714PA_ug','1v-gfO-1BBrS94mE64175BTeN68LaP7Ua','1OsXQuV0gy0qC4ri1qLXpRGbwYxA7slm_','1qJMvgNXDOQOIlR5EO0-gFsYnJJzdRIxi','1Wff-_HYEjqc0bWM7Rgrv-7TlfPlbJD9m','1HQT33xO7amR7AfMLY55gUnW4bunKEcbW','1Uu48ZkF0KQMt1BOppiEbLyFZz5ABuJkj','1Hih7FW6_mAyrT8EwSSGPXDYBHbbeq1m_','1xlGeRm7lLFC0updZoNHxsyaEkax-8uu_','15-W-zTCX_VZkLbUUnVMC755qolZ7iYXe','18z2GHPszz1A4C7ViXCNHH0R9Sqtfjt_k','1-xraPAAnNsJcWvN00VJInrfNuPX3DC57','1lrn6XqiFf_APDJvm_wVLs5jLHWDRXUx6','1Xz0jLd8dkUiifOFEFvCxGsnsF1N07zOr']
i = 0
df_list = []
for id in ids:
  #print('Principio iteración' + str(i+1))
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('CIC-IDS-2018-' + str(i) + '.csv') 
  df = pd.read_csv('CIC-IDS-2018-' + str(i) + '.csv')
  df_list.append(df)
  #print('Final iteración' + str(i+1))
  i = i + 1
df = pd.concat(df_list)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Label"] = le.fit_transform(df["Label"])
df

In [ ]:
df.astype({'Timestamp': 'float64'}).dtypes

In [ ]:
df.drop_duplicates()

In [ ]:
#Dividir el dataset en datos y clase a predecir
#X = df.drop(['Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 'Fwd IAT Std', 'Bwd IAT Mean', 'Bwd IAT Std', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'Avg Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Active Mean', 'Active Std', 'Idle Mean', 'Idle Std', 'Label']
#            , axis=1)
X = df.drop(['Timestamp','Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 'Fwd IAT Std', 'Bwd IAT Mean', 'Bwd IAT Std', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'Avg Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Active Mean', 'Active Std', 'Idle Mean', 'Idle Std', 'Label', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Fwd PSH Flags', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Max', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'Total Backward Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Min', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Total', 'Bwd IAT Max', 'Fwd PSH Flags', 'Fwd Header Length', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Min', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min']
            , axis=1)
y = df['Label']

# One-hot encoding
#y = pd.get_dummies(df.Label)

In [ ]:
data=pd.concat([y, X],  axis=1)

In [ ]:
df['Label'].value_counts()

6     575364
0     503841
1     433605
4     198861
8     145199
12    127844
7      41406
10      9908
5       1730
2        551
3        228
13        78
9         55
11        41
Name: Label, dtype: int64

In [ ]:
data.columns.size

In [ ]:
data.to_csv('default.csv',sep = ',', index = False, header = None)

In [ ]:
data.columns.get_loc("Benign")

In [ ]:
data.columns.get_loc("SQL Injection")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import model_search
from model_search import constants
from model_search import single_trainer
from model_search.data import csv_data

In [ ]:
!ls model_search/configs/

In [ ]:
#[0,1,2,3,4,5,6,7,8,9,10,11,12,13]
trainer = single_trainer.SingleTrainer(
    data=csv_data.Provider(label_index=0, record_defaults=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],logits_dimension=14, filename="default.csv"),
    spec='model_search/configs/dnn_config.pbtxt')

In [ ]:
!mkdir /tmp/run_example_4

In [ ]:
import sys
from absl import app

# Addresses `UnrecognizedFlagError: Unknown command line flag 'f'`
sys.argv = sys.argv[:1]

# `app.run` calls `sys.exit`
try:
  app.run(lambda argv: None)
except:
  pass

In [ ]:
!mkdir /tmp/run_example_4

In [ ]:
trainer.try_models(
    number_models=100,
    train_steps=50,
    eval_steps=2,
    root_dir="/tmp/run_example_4",
    batch_size=512,
    experiment_name="example4",
    experiment_owner="model_search_4")